# Notebook to demo the usage of MCPAgent.

In this notebook we are lanching 2 MCP servers via [docker](https://www.docker.com/). One to get the time and the second to enable file system access.

We will ask the agent to create a file with the local current time.

Make sure to docker pull the models before executing the following cells:

```bash
docker pull mcp/time
docker pull mcp/filesystem
```

## References
- https://hub.docker.com/r/mcp/time
- https://hub.docker.com/r/mcp/filesystem

```json
{
  "mcpServers": {
    "time": {
      "command": "docker",
      "args": [
        "run",
        "-i",
        "--rm",
        "mcp/time"
      ]
    }
  }
}
```

```json
{
  "mcpServers": {
    "filesystem": {
      "command": "docker",
      "args": [
        "run",
        "-i",
        "--rm",
        "-v",
        "/local-directory:/local-directory",
        "mcp/filesystem",
        "/local-directory"
      ]
    }
  }
}
```

In [1]:
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=UserWarning)

## Import the usual suspects.

In [2]:
import logging
import os
import shutil
from contextlib import AsyncExitStack

from gait import AMAO, MCPAgent, MCPServerStdio, AObserverLoguru
from rich.pretty import pprint

## Get the full path of the docker command.

In [3]:
docker = shutil.which("docker")
assert docker, "docker not found"
print(docker)

/Users/mraad/.local/bin/docker


## Create the `mcp_filesystem` folder in the home directory.

In [4]:
mcp_filesystem_path = os.path.expanduser(os.path.join("~", "mcp_filesystem"))
os.makedirs(mcp_filesystem_path, exist_ok=True)

## Let's ask question.

In [5]:
async with AsyncExitStack() as stack:
    # Launch the MCP time server.
    mcp_time = await stack.enter_async_context(
        MCPServerStdio(
            command=docker,
            args=[
                "run",
                "-i",
                "--rm",
                "mcp/time",
            ],
        )
    )
    # Launch the MCP filesystem server.
    mcp_file = await stack.enter_async_context(
        MCPServerStdio(
            command=docker,
            args=[
                "run",
                "-i",
                "--rm",
                # Note how we are mapping a volume from the "outside" to the "inside"
                "-v",
                f"{mcp_filesystem_path}:/mcp_filesystem",
                "mcp/filesystem",
                "/mcp_filesystem",
            ],
        )
    )
    # Create the agent.
    async with MCPAgent(
        model="azure/" + os.environ["AZURE_API_DEPLOYMENT"],
        # Reference the servers.
        mcp_servers=[
            mcp_time,
            mcp_file,
        ],
    ) as agent:
        # Start the orchestration with an observer.
        mao = AMAO(
            agent,
            observer=AObserverLoguru(),
        )
        async for _ in mao(
            "Create a file named current_time.txt and write in it the current time in Atlanta, GA.",
        ):
            # Terminate the loop when the file is created.
            if _.content:
                mao.terminate()

07:27:05 | INFO | GAIT started.
07:27:05 | INFO | Iteration 1 (MCPAgent-azure/gpt-4.1) started.
07:27:07 | INFO | Content: None
07:27:07 | INFO | Function: get_current_time({"timezone":"America/New_York"})
07:27:07 | INFO | Observation: {
  "timezone": "America/New_York",
  "datetime": "2025-06-21T07:27:07-04:00",
  "is_dst": true
}
07:27:07 | INFO | Iteration 2 (MCPAgent-azure/gpt-4.1) started.
07:27:07 | INFO | Content: None
07:27:07 | INFO | Function: write_file({"path":"current_time.txt","content":"Current time in Atlanta, GA: 2025-06-21 07:27:07 EDT"})
07:27:07 | INFO | Observation: Error calling tool write_file
Error: Access denied - path outside allowed directories: /app/current_time.txt not in /mcp_filesystem
07:27:07 | INFO | Iteration 3 (MCPAgent-azure/gpt-4.1) started.
07:27:08 | INFO | Content: None
07:27:08 | INFO | Function: write_file({"path":"/mcp_filesystem/current_time.txt","content":"Current time in Atlanta, GA: 2025-06-21 07:27:07 EDT"})
07:27:08 | INFO | Observatio

## Let's see the content of the file.

In [6]:
file_name = os.path.join(mcp_filesystem_path, "current_time.txt")
if os.path.exists(file_name):
    with open(file_name, mode="r") as fp:
        print(fp.read())
else:
    print(f"{file_name} was not created :-(")

Current time in Atlanta, GA: 2025-06-21 07:27:07 EDT
